In [2]:
import numpy as np

# Candidate Elimination Algorithm
def candidate_elimination(data, target):
    # Number of attributes
    num_attributes = data.shape[1]
    
    # Initialize the most specific hypothesis (S) and the most general hypothesis (G)
    S = ['Ø'] * num_attributes  # Start with the null hypothesis (most specific)
    G = [['?'] * num_attributes]  # Start with the most general hypothesis

    # Iterate through training examples
    for i, example in enumerate(data):
        if target[i] == "Yes":  # Positive example
            # Generalize S minimally to include the positive example
            for j in range(num_attributes):
                if S[j] == 'Ø':  # Update first positive example
                    S[j] = example[j]
                elif S[j] != example[j]:  # If not consistent, generalize with '?'
                    S[j] = '?'
            # Remove hypotheses in G inconsistent with the positive example
            G = [g for g in G if is_consistent(g, example)]
        
        elif target[i] == "No":  # Negative example
            # Remove hypotheses in S inconsistent with the negative example
            if is_consistent(S, example):
                S = ['Ø'] * num_attributes  # Reset S if inconsistent
            
            # Specialize G minimally to exclude the negative example
            new_G = []
            for g in G:
                if is_consistent(g, example):
                    for j in range(num_attributes):
                        if g[j] == '?':  # General condition, specialize minimally
                            for value in np.unique(data[:, j]):
                                if value != example[j]:
                                    new_hypothesis = g.copy()
                                    new_hypothesis[j] = value
                                    if is_more_general(new_hypothesis, S):
                                        new_G.append(new_hypothesis)
                else:
                    new_G.append(g)
            G = new_G

    return S, G


# Helper Functions
def is_consistent(hypothesis, example):
    """Check if a hypothesis is consistent with a given example."""
    for h, e in zip(hypothesis, example):
        if h != '?' and h != e:
            return False
    return True

def is_more_general(h1, h2):
    """Check if hypothesis h1 is more general than h2."""
    more_general = False
    for x, y in zip(h1, h2):
        if x == '?' or x == y:
            continue
        elif y == 'Ø':
            continue
        elif x != y:
            return False
        more_general = True
    return more_general

# Example Dataset
data = np.array([
    ['Sunny', 'Warm', 'Normal', 'Strong', 'Warm', 'Same'],
    ['Sunny', 'Warm', 'High', 'Strong', 'Warm', 'Same'],
    ['Rainy', 'Cold', 'High', 'Strong', 'Warm', 'Change'],
    ['Sunny', 'Warm', 'High', 'Strong', 'Cool', 'Change']
])
target = np.array(['Yes', 'Yes', 'No', 'Yes'])

# Run Candidate Elimination Algorithm
S, G = candidate_elimination(data, target)

# Display Results
print("Final Specific Hypothesis (S):", S)
print("Final General Hypotheses (G):", G)


Final Specific Hypothesis (S): [np.str_('Sunny'), np.str_('Warm'), '?', np.str_('Strong'), '?', '?']
Final General Hypotheses (G): []
